In [ ]:
!pip install opendatasets
!pip install utils
!pip install livelossplot

In [ ]:
import opendatasets as od
od.download("https://www.kaggle.com/paultimothymooney/chest-xray-pneumonia")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: 

**Import needed Librabries :**

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import utils
import pandas as pd
import os
from os import listdir, makedirs, getcwd, remove
from os.path import isfile, join, abspath, exists, isdir, expanduser
from PIL import Image
from pathlib import Path
from skimage.io import imread

%matplotlib inline

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Input, Dropout,Flatten, Conv2D
from tensorflow.keras.layers import BatchNormalization, Activation, MaxPooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.utils import plot_model

from tensorflow.keras.applications import resnet50

from IPython.display import SVG, Image
from livelossplot import PlotLossesKerasTF
import tensorflow as tf
print("Tensorflow version:", tf.__version__)

In [ ]:
xray_train_prop=[]

for image in os.listdir("./chest-xray-pneumonia/chest_xray/train/"):
    xray_train_prop.append(len(os.listdir("./chest-xray-pneumonia/chest_xray/train/" + image)))
    print(str(len(os.listdir("./chest-xray-pneumonia/chest_xray/train/" + image))) + " " + image + " images")

print("xray_train_prop = ",xray_train_prop)

In [ ]:
xray_train = ['PNEUMONIA','NORMAL']
colors = ['lightgreen','cornflowerblue']
plt.figure(figsize=[12,6])
plt.bar(x=xray_train, height=xray_train_prop, color=colors, edgecolor='black')
plt.xlabel('Xray_ class')
plt.ylabel('Proportion')
plt.title('Xray Label Proportions')
plt.show()

We could see that the number of images in the PNEUMONIA class are more than that of the NORMAL class by the double. so we will do oversampling for the normal class.

In [ ]:
train_dir= Path("./chest-xray-pneumonia/chest_xray/train")

# Get the path to the normal and pneumonia sub-directories
normal_cases_dir = train_dir / 'NORMAL'
pneumonia_cases_dir = train_dir / 'PNEUMONIA'

# Get the list of all the images
normal_cases = normal_cases_dir.glob('*.jpeg')
pneumonia_cases = pneumonia_cases_dir.glob('*.jpeg')

# An empty list. We will insert the data into this list in (img_path, label) format
train_dataset = []

# Go through all the normal cases. The label for these cases will be 0
for img in normal_cases:
    train_dataset.append((img,0))

# Go through all the pneumonia cases. The label for these cases will be 1
for img in pneumonia_cases:
    train_dataset.append((img, 1))

# Get a pandas dataframe from the data we have in our list 
train_dataset = pd.DataFrame(train_dataset, columns=['image', 'label'],index=None)

# Shuffle the data 
train_dataset = train_dataset.sample(frac=1.).reset_index(drop=True)

# How the dataframe looks like?
train_dataset.head()

In [ ]:
pneumonia_samples = (train_dataset[train_dataset['label']==1]['image'].iloc[:3]).tolist()
normal_samples = (train_dataset[train_dataset['label']==0]['image'].iloc[:3]).tolist()

f, ax = plt.subplots(2,3, figsize=(30,15))
for i in range(3):
    img = imread(pneumonia_samples[i])
    ax[0, i%3].imshow(img, cmap='gray')
    ax[0, i%3].set_title("Pneumonia")
    img = imread(normal_samples[i])
    ax[1, i%3].imshow(img, cmap='gray')
    ax[1, i%3].set_title("Normal")
    ax[0, i%3].axis('off')
    ax[1, i%3].axis('off')
    ax[0, i%3].set_aspect('auto')
    ax[1, i%3].set_aspect('auto')
plt.show()

Applying CNN Architecture:

In [ ]:
img_size = 224
batch_size = 32

In [ ]:
datagen_train = ImageDataGenerator(rotation_range=15)
train_generator = datagen_train.flow_from_directory("./chest-xray-pneumonia/chest_xray/train/",
                                                    target_size=(img_size,img_size),
                                                    color_mode="rgb",
                                                    batch_size=batch_size,
                                                    class_mode='binary',
                                                    shuffle=True)

datagen_validation = ImageDataGenerator(rotation_range=15)
validation_generator = datagen_validation.flow_from_directory("./chest-xray-pneumonia/chest_xray/test/",
                                                    target_size=(img_size,img_size),
                                                    color_mode="rgb",
                                                    batch_size=batch_size,
                                                    class_mode='binary',
                                                    shuffle=True)


In [ ]:
# Initialising the CNN
model = Sequential()

# 1 - Convolution
model.add(Conv2D(64,(3,3), padding='same', input_shape=(img_size, img_size,3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# 2nd Convolution layer
model.add(Conv2D(128,(5,5), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# 3rd Convolution layer
model.add(Conv2D(512,(3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# 4th Convolution layer
model.add(Conv2D(512,(3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

#  Flattening
model.add(Flatten())

# Fully connected layer 1st layer
model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

# Fully connected layer 2nd layer
model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(1, activation='softmax'))

In [ ]:
model.compile(optimizer=Adam(lr=0.0005), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

**Train and Evaluate the Model:**

In [ ]:
epochs = 50
steps_per_epoch = train_generator.n//train_generator.batch_size
validation_steps = validation_generator.n//validation_generator.batch_size

#Reduce learning rate when a metric has stopped improving.
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                              patience=2, min_lr=0.00001, mode='auto')
#save the Keras model or model weights at some frequency.
checkpoint = ModelCheckpoint("model_weights5.h5", monitor='val_accuracy',
                             save_weights_only=True, mode='max', verbose=1)
#Stop training when a monitored metric has stopped improving.
early_stopp = tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
#Callback
callbacks1 = [PlotLossesKerasTF(), checkpoint, reduce_lr,early_stopp]

In [ ]:
history = model.fit(x=train_generator,
                      steps_per_epoch=steps_per_epoch,
                      epochs=epochs,
                      validation_data = validation_generator,
                      validation_steps = validation_steps,
                      callbacks=callbacks1)

In [ ]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

ResNet 50 Model:

In [ ]:
resnet50_model = resnet50.ResNet50(include_top=True,
                               weights="imagenet",
                               input_shape=(img_size,img_size,3),
                               classifier_activation="softmax",
                               classes=1000)

In [ ]:
### # Initialising the CNN
model2 = Sequential()
model2.add(resnet50_model) 

# Fully connected layer 1st layer
model2.add(Dense(256))
model2.add(BatchNormalization())
model2.add(Activation('relu'))
model2.add(Dropout(0.25))

# Fully connected layer 2nd layer
model2.add(Dense(512))
model2.add(BatchNormalization())
model2.add(Activation('relu'))
model2.add(Dropout(0.25))

model2.add(Dense(1, activation='softmax'))

In [ ]:
model2.compile(optimizer=Adam(lr=0.0005), loss='categorical_crossentropy', metrics=['accuracy','Recall','Precision','AUC'])
model2.summary()

In [ ]:
epochs = 15
steps_per_epoch = train_generator.n//train_generator.batch_size
validation_steps = validation_generator.n//validation_generator.batch_size

#Reduce learning rate when a metric has stopped improving.
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                              patience=2, min_lr=0.00001, mode='auto')
#save the Keras model or model weights at some frequency.
checkpoint = ModelCheckpoint("model_weights2.h5", monitor='val_accuracy',
                             save_weights_only=True, mode='max', verbose=1)
#Stop training when a monitored metric has stopped improving.
early_stopp = tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
#Callback
callbacks1 = [PlotLossesKerasTF(), checkpoint, reduce_lr,early_stopp]

In [ ]:
history = model2.fit(x=train_generator,
                      steps_per_epoch=steps_per_epoch,
                      epochs=epochs,
                      validation_data = validation_generator,
                      validation_steps = validation_steps,
                      callbacks=callbacks1)

In [ ]:
model_json = model2.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)